# Imports

In [ ]:
# Standard Library
import os

# External
# from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# ibmpairs
import ibmpairs.authentication as authentication
import ibmpairs.catalog as catalog
import ibmpairs.client as client
import ibmpairs.query as query
import ibmpairs.dashboard as dashboard

# Authenticate

## OAuth2

#### In order to authenticate to IBM Environmental Intelligence: Geospatial APIs (formerly IBM PAIRS Geoscope) using the ibmpairs library; an API_KEY, TENANT_ID (or alternatively CLIENT_ID) and an ORG_ID are required.

In [ ]:
EI_API_KEY   = os.environ.get('EI_API_KEY')
EI_TENANT_ID = os.environ.get('EI_TENANT_ID')
EI_ORG_ID    = os.environ.get('EI_ORG_ID')

# credentials   = authentication.OAuth2(api_key   = EI_API_KEY,
#                                       tenant_id = EI_TENANT_ID,
#                                       org_id    = EI_ORG_ID)

# If you have a client id and not a tenant_id:
# EI_CLIENT_ID = os.environ.get('EI_CLIENT_ID')
# credentials  = authentication.OAuth2(api_key   = EI_API_KEY,
#                                      client_id = EI_CLIENT_ID,
#                                      org_id    = EI_ORG_ID)

#### If you are a user of the legacy IBM PAIRS Geoscope system and wish to authenticate to pairs.res.ibm.com, please see Legacy -> Authenticate -> OAuth2/Basic below.

# Client

#### A general client is created with the authentication type; the client will become a global default once declared.

In [ ]:
# ei_client = client.Client(authentication = credentials)

#### There is a help method which wraps the aforementioned Authenticate and Client steps that can be used alternatively.

In [ ]:
ei_client = client.get_client(api_key   = EI_API_KEY,
                              tenant_id = EI_TENANT_ID,
                              org_id    = EI_ORG_ID)

# Query

## Point

#### An example query to gather information about a spatial point.

In [ ]:
point_dict = {
  "layers": [
    {
      "id": "49429",
      "type": "raster"
    }
  ],
  "spatial": {
    "coordinates": [
      "35.6523",
      "-87.62"
    ],
    "type": "point"
  },
  "temporal": {
    "intervals": [
      {
        "start": "2023-01-01T00:00:00Z",  
        "end": "2023-01-01T23:59:59Z"
      }
    ]
  }
}

In [ ]:
point_query = query.submit(point_dict)

In [ ]:
point_df = point_query.point_data_as_dataframe()

In [ ]:
dfpivot = point_df.pivot(index=['timestamp','latitude','longitude'], columns='layer_id',values=['value'])

In [ ]:
dfpivot

In [ ]:
# Analyse the returned data.

## Raster

#### An example raster query.

In [ ]:
raster_dict = {
  "layers": [
    {
      "id": "49429",
      "type": "raster"
    }
  ],
  "spatial": {
    "coordinates": [
      "35.6523",
      "-87.62",
      "51.6523",
      "-71.62"
    ],
    "type": "square"
  },
  "temporal": {
    "intervals": [
      {
        "start": "2023-01-01T00:00:00Z",
        "end":   "2023-01-01T00:00:00Z"
      }
    ]
  }
}

In [ ]:
raster_query = query.submit_check_status_and_download(raster_dict)

In [ ]:
raster_files = raster_query.list_files()
print(raster_files)

In [ ]:
# Analyse the returned data.

# Replace Dates

#### The replace dates method replaces the all of the dates within a query object and sets a new nickname (optional), this is useful if a query object exists and the desire is to re-run with different date ranges.

In [ ]:
from datetime import datetime
start = datetime(2023,1,2,0,0,0)
end   = datetime(2023,1,2,0,0,0)

print()

raster_query_new_dates = query.query_from_dict(raster_dict)
raster_query_new_dates.replace_dates(start_date = start,
                                     end_date   = end)

print("")
print(raster_query_new_dates)

## Batch Query (Async)

#### Execute a number of queries asynchronously according to a number of async workers.

In [ ]:
from datetime import datetime
start = datetime(2023,1,2,0,0,0)
end   = datetime(2023,1,2,0,0,0)

raster_query_batch          = query.query_from_dict(raster_dict)
raster_query_batch_next_day = query.query_from_dict(raster_dict)
raster_query_batch_next_day.replace_dates(start_date = start,
                                          end_date   = end)
queries = [raster_query_batch, raster_query_batch_next_day]

result = query.batch_query(queries = queries,
                           workers = 2,
                           verify = False)

print(result)

# Catalog

## Search

#### Search IBM Environmental Intelligence: Geospatial APIs catalog (Datasets, Datalayers) for an id or term.

In [ ]:
search_by_id = catalog.search("49464")
search_by_id

In [ ]:
search_by_word = catalog.search("Sentinel")
search_by_word

## Get a list of Datasets

#### Gather all Datasets a user has access to.

In [ ]:
ds_list = catalog.get_data_sets()
ds_list.display()

## Get a Dataset

#### Get a Dataset by an ID number 

In [ ]:
ds = catalog.get_data_set(id = "177")
ds.display()

## Get a list of Datalayers

#### Gather all Datalayers a user has access to.

In [ ]:
dl_list = catalog.get_data_layers()
dl_list.display()

## Get a list of Datalayers per Dataset

#### Gather all Datalayers in a given Dataset by ID number.

In [ ]:
dl_list_by_set = catalog.get_data_layers(data_set_id = "177")
dl_list_by_set.display()

## Get a Datalayer

#### Get a Datalayer by an ID number 

In [ ]:
dl = catalog.get_data_layer(id = "49464")
dl.display()

## Get a list of Datalayer Dimensions per Datalayer

#### Gather all Datalayer Dimensions in a given Datalayer by ID number.

In [ ]:
dlds = catalog.get_data_layer_dimensions(data_layer_id = "49166")
dlds.display()

## Get a Datalayer Dimension

#### Get a Datalayer Dimension by an ID number 

In [ ]:
dld = catalog.get_data_layer_dimension(id = "243")
dld.display()

# Legacy

## Authenticate

### Basic

In [ ]:
EI_USERNAME = os.environ.get('EI_USERNAME')
EI_PASSWORD = os.environ.get('EI_PASSWORD')

credentials  = authentication.Basic(username = EI_USERNAME,
                                    password = EI_PASSWORD)

### OAuth2

In [ ]:
EI_API_KEY = os.environ.get('EI_API_KEY')

credentials = authentication.OAuth2(api_key = EI_API_KEY)

## Query

### Vector

#### An example vector query.

In [ ]:
vector_dict = {
  "layers": [
    {
      "id": "P265C4176",
      "type": "vector"
    }
  ],
  "outputType": "csv",
  "publish": False,
  "spatial": {
    "aoi": 24,
    "type": "poly"
  },
  "temporal": {
    "intervals": [
      {
        "end": "2018-11-14T00:00:00Z",
        "start": "2018-11-13T00:00:00Z"
      }
    ]
  }
}

In [ ]:
vector_query = query.submit_check_status_and_download(vector_dict)

In [ ]:
raster_files = raster_query.list_files()
print(raster_files)

In [ ]:
# Analyse the returned data.